<a href="https://colab.research.google.com/github/mbewustanley/yt_insights/blob/main/6_experiment_5_xgboost_with_hpt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install mlflow boto3 awscli optuna xgboost imbalanced-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.0/40.0 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 54.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 83.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.4/14.4 MB 56.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.7/404.7 kB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 570.5/570.5 kB 34.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.8/76.

In [3]:
import optuna # a hyperparameter optimization framework for ml
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import mlflow
import mlflow.sklearn

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE

In [4]:
#set mlflow tracking and experiment
mlflow.set_tracking_uri("http://ec2-13-244-94-213.af-south-1.compute.amazonaws.com:5000")
mlflow.set_experiment('Exp 5 - ML Algos with HP Tuning')

<Experiment: artifact_location='s3://stanley-mlflow-bucket-27/791009430485705644', creation_time=1764443100072, experiment_id='791009430485705644', last_update_time=1764443100072, lifecycle_stage='active', name='Exp 5 - ML Algos with HP Tuning', tags={'mlflow.experimentKind': 'custom_model_development'}>

In [6]:
# load data
df = pd.read_csv('/content/reddit_preprocessing.csv').dropna(subset=['clean_comment'])

In [7]:
# remap the class labels from [-1,0,1] to [2,0,1]. xgboost does not take negatove values
df['category'] = df['category'].map({-1:2, 0:0, 1:1})

In [8]:
# remove rows where target labels (category) are NAN
df = df.dropna(subset=['category'])

In [9]:
ngram_range = (1,3) #trigram settings
max_features = 1000  # set max features for TF-IDF

In [10]:
# train-test split before vectorization and resampling
X_train, X_test, y_train, y_test = train_test_split(df['clean_comment'], df['category'], test_size=0.2, random_state=42)


In [11]:
#vectorization using TF-IDF, fit on training data
vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=max_features)
X_train_vec = vectorizer.fit_transform(X_train) # fit on training data
X_test_vec = vectorizer.transform(X_test) # transform test data

In [12]:
smote = SMOTE(random_state=42)
X_train_vec, y_train = smote.fit_resample(X_train_vec, y_train)

In [13]:
# function to log results in MLFLOW
def log_mlflow(model_name, model, X_train, X_test, y_train, y_test):
    with mlflow.start_run():
        #log model type
        mlflow.set_tag('mlflow.runName', f"{model_name}_SMOTE_TFIDF_Trigrams")
        mlflow.set_tag('experiment_type', "algorithm_comparison")

        # log algorithm name as a parameter
        mlflow.log_param("algo_name", model_name)

        # train model
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        #log accuracy
        accuracy= accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        #log classification report
        classification_rep =classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        #log the model
        mlflow.sklearn.log_model(model, f"{model_name}_model")

In [14]:
#optuna objective function for xgboost
def objective_xgboost(trial):
    n_estimators = trial.suggest_int('n_estimators', 50, 300)
    learning_rate = trial.suggest_float('learning_rate', 1e-4, 1e-1, log=True)
    max_depth = trial.suggest_int('max_depth', 3, 10)

    model = XGBClassifier(n_estimators-n_estimators, learning_rate=learning_rate, max_depth=max_depth, random_State=42)
    return accuracy_score(y_test, model.fit(X_train_vec, y_train).predict(X_test_vec))



In [15]:
# run optuna for XGBoost, log the best model only
def run_optuna_experiment():
    study = optuna.create_study(direction='maximize')
    study.optimize(objective_xgboost, n_trials=30)

    #get the best parameters and log only the best model
    best_params = study.best_params
    best_model = XGBClassifier(n_estimators=best_params['n_estimators'], learning_rate=best_params['learning_rate'], max_depth = best_params['max_depth'], random_state=42)

    #log the nest model with mlflow, passing the algo_name as 'xgboost'
    log_mlflow("XGBoost", best_model, X_train_vec, X_test_vec, y_train, y_test)


In [16]:
#run the experiment for xgboost
run_optuna_experiment()

[I 2025-12-01 12:54:50,500] A new study created in memory with name: no-name-6df1d601-f053-4bf4-9c33-539e0e6821a0
/usr/local/lib/python3.12/dist-packages/xgboost/core.py:771: FutureWarning: Pass `objective` as keyword args.
  warnings.warn(msg, FutureWarning)
/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [12:54:50] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "random_State" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
[I 2025-12-01 12:55:44,605] Trial 0 finished with value: 0.5085231146870313 and parameters: {'n_estimators': 236, 'learning_rate': 0.00023121470929446318, 'max_depth': 4}. Best is trial 0 with value: 0.5085231146870313.
/usr/local/lib/python3.12/dist-packages/xgboost/core.py:771: FutureWarning: Pass `objective` as keyword args.
  warnings.warn(msg, FutureWarning)
/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [12:55:44] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "random_St

🏃 View run XGBoost_SMOTE_TFIDF_Trigrams at: http://ec2-13-244-94-213.af-south-1.compute.amazonaws.com:5000/#/experiments/791009430485705644/runs/78ee29039d5c4a96a962a0e07a45aade
🧪 View experiment at: http://ec2-13-244-94-213.af-south-1.compute.amazonaws.com:5000/#/experiments/791009430485705644
